In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.chrome.options import Options  


In [2]:
chrome_options = Options()  
chrome_options.add_argument("--headless")  
chrome_options.add_argument("--no-sandbox")

driver = webdriver.Chrome('chromedriver', options=chrome_options)

In [3]:
url = 'https://bnonews.com/index.php/2020/02/the-latest-coronavirus-cases/'
driver.get(url)
content = driver.page_source


### Extract date time

In [4]:
soup =BeautifulSoup(content, 'lxml')

In [5]:
data = soup.findAll(text=re.compile("Last update: .+"))
data

['Last update: 18 February 2020 at 12:07 a.m. ET']

In [6]:
m = re.search("Last update: (\d+ \w+ 202\d) at (\d+:\d+ [apAP].*[mM])", data[0])
data_date_time = ' '.join([m.group(1), m.group(2)])
data_date_time = re.sub("[\.]","",data_date_time)
data_date_time = datetime.strptime(data_date_time, '%d %B %Y %I:%M %p').strftime('%Y-%m-%d %H:%M')
data_date_time

'2020-02-18 00:07'

### Dataframes

In [7]:

dfs = pd.read_html(content)

In [8]:
len(dfs)
dfs[1]

,0,1,2,3,4
0,OTHER PLACES,Cases,Deaths,Notes,Links
1,Diamond Princess,454,0,"19 serious, 17 recovered",Source
2,Singapore,77,0,"4 critical, 24 recovered",Source
3,Japan,66*,1,1 serious,Source
4,Hong Kong,60,1,"5 critical, 2 serious, 1 recovered",Source
5,Thailand,35,0,"2 serious, 15 recovered",Source
6,South Korea,31,0,12 recovered,Source
7,Malaysia,22,0,8 recovered,Source
8,Taiwan,22,1,2 recovered,Source
9,Australia,15,0,10 recovered,Source


### International

In [9]:

df_intl =dfs[1]
df_intl.columns=['international', 'cases', 'd1', 'd2', 'd3']
df_intl

,international,cases,d1,d2,d3
0,OTHER PLACES,Cases,Deaths,Notes,Links
1,Diamond Princess,454,0,"19 serious, 17 recovered",Source
2,Singapore,77,0,"4 critical, 24 recovered",Source
3,Japan,66*,1,1 serious,Source
4,Hong Kong,60,1,"5 critical, 2 serious, 1 recovered",Source
5,Thailand,35,0,"2 serious, 15 recovered",Source
6,South Korea,31,0,12 recovered,Source
7,Malaysia,22,0,8 recovered,Source
8,Taiwan,22,1,2 recovered,Source
9,Australia,15,0,10 recovered,Source


In [10]:
df_intl.columns = df_intl.columns.map(str.lower)


#### Cases

In [11]:
df_intl_cases = df_intl[['international', 'cases']].set_index('international').T
df_intl_cases = df_intl_cases.apply(lambda row: 
                   [re.sub('[^\d]+', '', x)  for x in row ])
df_intl_cases = df_intl_cases.apply(lambda row: 
                   [int(x) if x!='' else 0  for x in row ])

In [12]:
df_intl_cases = df_intl_cases.drop(['TOTAL'], axis=1)

In [13]:
# add date time column
df_intl_cases.insert(0, 'datetime', data_date_time)
df_intl_cases

international,datetime,OTHER PLACES,Diamond Princess,Singapore,Japan,Hong Kong,Thailand,South Korea,Malaysia,Taiwan,...,Italy,Russia,Spain,Nepal,Cambodia,Sri Lanka,Finland,Sweden,Belgium,Egypt
cases,2020-02-18 00:07,0,454,77,66,60,35,31,22,22,...,3,2,2,1,1,1,1,1,1,1


#### Save to file

In [14]:
from pathlib import Path
csv_file = 'data/bnonews-international.csv'

df = df_intl_cases

if Path(csv_file).exists():
    old_df = pd.read_csv(csv_file, parse_dates=True)

    df = pd.concat([old_df, df], sort=True)
df = df.set_index('datetime').fillna(0)

In [15]:

df = df.loc[~df.index.duplicated(keep='first')]
df.sort_values(by=['datetime'], inplace=True)


for col in df.columns:
    df[col] = df[col].astype('int')
    
df

,Australia,Belgium,Cambodia,Canada,Diamond Princess,Egypt,Finland,France,Germany,Hong Kong,...,South Korea,Spain,Sri Lanka,Sweden,Taiwan,Thailand,UAE,United Kingdom,United States,Vietnam
datetime,,,,,,,,,,,,,,,,,,,,,
2020-02-06 08:03,15,1,1,5,0,0,1,6,12,0,...,23,1,1,1,0,25,5,2,12,12
2020-02-06 08:19,15,1,1,5,0,0,1,6,13,0,...,23,1,1,1,0,25,5,3,12,12
2020-02-06 09:01,15,1,1,5,0,0,1,6,13,0,...,23,1,1,1,0,25,5,3,12,12
2020-02-06 09:46,15,1,1,5,0,0,1,6,13,0,...,23,1,1,1,0,25,5,3,12,12
2020-02-06 17:56,15,1,1,7,0,0,1,6,13,0,...,23,1,1,1,0,25,5,3,12,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-02-17 17:49,15,1,1,8,454,1,1,12,16,60,...,30,2,1,1,22,35,9,9,15,16
2020-02-17 19:07,15,1,1,8,454,1,1,12,16,60,...,30,2,1,1,22,35,9,9,15,16
2020-02-17 21:05,15,1,1,8,454,1,1,12,16,60,...,31,2,1,1,22,35,9,9,15,16


In [16]:

df.to_csv(csv_file, index=True)


Text(0.5, 0, 'Number of cases, sampled at 2020-02-18 00:07')

In [17]:
import seaborn as sns

series = df.iloc[-1]

sns.set(rc={'figure.figsize':(14,9)})
ax = sns.barplot(y=series.index, x=series)
ax.set_xlabel('Number of cases, sampled at '+series.name)



In [18]:
png_file='images/bnonews-international.png'
fig = ax.get_figure()
fig.savefig(png_file)